In [1]:
# Add confidence interval
import config_cat_embedding
import pandas as pd
import numpy as np
import random
import time
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
from xgboost import XGBClassifier
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score)
from scipy import stats

from data_prep import bank_data_prep, adult_data_prep
from embedding_helper import create_network

# Set the random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Load and preprocess data
data_path = config_cat_embedding.paths['data']
bank_data = pd.read_csv(data_path+'adult.csv', sep=',')

df_bank, cat_cols = adult_data_prep(bank_data)

X = df_bank.iloc[:, :-1]
y = df_bank.y

# Define the classifiers
seed = 42
# We will determine the number_of_features inside the cross-validation loop after preprocessing

models = [
    ('LR', LogisticRegression(solver='lbfgs', random_state=seed, max_iter=1000)),
    ('DT', DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=seed)),
    ('RF', RandomForestClassifier(n_estimators=200, max_depth=5, random_state=seed, min_samples_leaf=3)),
    ('KNN', KNeighborsClassifier(n_neighbors=3)),
    ('XGB', XGBClassifier(eval_metric='logloss', random_state=seed)),
    # ('SVM', SVC(gamma='scale', random_state=seed, probability=True)),
    ('MLP', KerasClassifier(
        model=create_network,
        epochs=100, batch_size=100, verbose=0, random_state=seed))
]

# Cross-validation setup
cv = StratifiedKFold(n_splits=20, shuffle=True, random_state=seed)

# Function to calculate confidence intervals
def confidence_interval(data, confidence=0.95):
    n = len(data)
    m = np.mean(data)
    std_err = stats.sem(data)
    h = std_err * stats.t.ppf((1 + confidence) / 2, n - 1)
    return m, m - h, m + h

# Loop over models
for name, model in models:
    print(f"Classifier: {name}")
    # Lists to store metrics for each fold
    accuracies = []
    precisions = []
    recalls = []
    f1s = []
    roc_aucs = []
    
    # Start the timer before cross-validation
    start_time = time.time()
    
    fold = 1
    for train_index, test_index in cv.split(X, y):
        # Split the data into training and testing sets for this fold
        X_train_fold, X_test_fold = X.iloc[train_index].copy(), X.iloc[test_index].copy()
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
        
        # Preprocess data within the fold
        # Target encoding
        import category_encoders as ce
        target_encoder = ce.TargetEncoder(cols=cat_cols, smoothing=1)
        X_train_enc = target_encoder.fit_transform(X_train_fold, y_train_fold)
        X_test_enc = target_encoder.transform(X_test_fold)
        
        # Standard scaling
        stc = StandardScaler()
        X_train_scaled = stc.fit_transform(X_train_enc)
        X_test_scaled = stc.transform(X_test_enc)
        
        # Update number_of_features for MLP
        number_of_features = X_train_scaled.shape[1]
        if name == 'MLP':
            # Update the model with the correct number of features
            model.set_params(model__number_of_features=number_of_features)
        
        # Fit the model
        model.fit(X_train_scaled, y_train_fold)
        # Predict on the test fold
        y_pred_fold = model.predict(X_test_scaled)
        
        # Get prediction probabilities for ROC AUC
        if hasattr(model, "predict_proba"):
            y_pred_prob_fold = model.predict_proba(X_test_scaled)[:, 1]
        else:
            # For classifiers without predict_proba, use decision_function
            y_pred_prob_fold = model.decision_function(X_test_scaled)
            # Normalize the decision function output to [0,1] range
            y_pred_prob_fold = (y_pred_prob_fold - y_pred_prob_fold.min()) / (y_pred_prob_fold.max() - y_pred_prob_fold.min())
        
        # Calculate metrics
        accuracies.append(accuracy_score(y_test_fold, y_pred_fold))
        precisions.append(precision_score(y_test_fold, y_pred_fold, zero_division=0))
        recalls.append(recall_score(y_test_fold, y_pred_fold))
        f1s.append(f1_score(y_test_fold, y_pred_fold))
        roc_aucs.append(roc_auc_score(y_test_fold, y_pred_prob_fold))
        
        # print(f"Fold {fold} completed.")
        fold += 1
    
    # Stop the timer after cross-validation
    end_time = time.time()
    total_computation_time = end_time - start_time  # Total time for the model
    
    # Calculate mean and confidence intervals
    acc_mean, acc_ci_lower, acc_ci_upper = confidence_interval(accuracies)
    prec_mean, prec_ci_lower, prec_ci_upper = confidence_interval(precisions)
    rec_mean, rec_ci_lower, rec_ci_upper = confidence_interval(recalls)
    f1_mean, f1_ci_lower, f1_ci_upper = confidence_interval(f1s)
    roc_mean, roc_ci_lower, roc_ci_upper = confidence_interval(roc_aucs)
    
    # Print results
    print(f"Accuracy: {acc_mean:.3f} (95% CI: {acc_ci_lower:.3f} - {acc_ci_upper:.3f})")
    print(f"Precision: {prec_mean:.3f} (95% CI: {prec_ci_lower:.3f} - {prec_ci_upper:.3f})")
    print(f"Recall: {rec_mean:.3f} (95% CI: {rec_ci_lower:.3f} - {rec_ci_upper:.3f})")
    print(f"F1 Score: {f1_mean:.3f} (95% CI: {f1_ci_lower:.3f} - {f1_ci_upper:.3f})")
    print(f"ROC AUC: {roc_mean:.3f} (95% CI: {roc_ci_lower:.3f} - {roc_ci_upper:.3f})")
    print(f"Total Computation Time: {total_computation_time:.3f} seconds\n")


Classifier: LR
Accuracy: 0.844 (95% CI: 0.841 - 0.847)
Precision: 0.739 (95% CI: 0.730 - 0.747)
Recall: 0.572 (95% CI: 0.562 - 0.581)
F1 Score: 0.644 (95% CI: 0.637 - 0.652)
ROC AUC: 0.899 (95% CI: 0.895 - 0.902)
Total Computation Time: 7.731 seconds

Classifier: DT
Accuracy: 0.840 (95% CI: 0.838 - 0.843)
Precision: 0.770 (95% CI: 0.763 - 0.777)
Recall: 0.508 (95% CI: 0.496 - 0.521)
F1 Score: 0.612 (95% CI: 0.602 - 0.621)
ROC AUC: 0.861 (95% CI: 0.857 - 0.865)
Total Computation Time: 8.595 seconds

Classifier: RF
Accuracy: 0.852 (95% CI: 0.848 - 0.855)
Precision: 0.813 (95% CI: 0.804 - 0.822)
Recall: 0.521 (95% CI: 0.510 - 0.532)
F1 Score: 0.635 (95% CI: 0.626 - 0.644)
ROC AUC: 0.906 (95% CI: 0.903 - 0.910)
Total Computation Time: 84.797 seconds

Classifier: KNN
Accuracy: 0.823 (95% CI: 0.819 - 0.828)
Precision: 0.657 (95% CI: 0.648 - 0.666)
Recall: 0.603 (95% CI: 0.590 - 0.616)
F1 Score: 0.629 (95% CI: 0.618 - 0.639)
ROC AUC: 0.830 (95% CI: 0.823 - 0.837)
Total Computation Time: 74.71

In [1]:
import config_cat_embedding
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from tqdm import tqdm_notebook as tqdm

from data_prep import bank_data_prep, adult_data_prep
from embedding_helper import create_network
import time
import random

# Set the random seed for reproducibility
random.seed(42)
np.random.seed(42)

data_path = config_cat_embedding.paths['data']
data_path_out = config_cat_embedding.paths['data_output']

bank_data = pd.read_csv(data_path+'adult.csv', sep=',')

df_bank, cat_cols = adult_data_prep(bank_data)


X = df_bank.iloc[:, :-1]
target = df_bank.y

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=1500)

# One-hot encoding
import category_encoders as ce
target_encoder = ce.TargetEncoder(cols=cat_cols,smoothing=1)
X_train_ohe = target_encoder.fit_transform(X_train, y_train)
X_test_ohe = target_encoder.transform(X_test)  # Use transform() instead of fit_transform()

# Standard scaling
stc = StandardScaler()
X_train_scaled = stc.fit_transform(X_train_ohe)
X_test_scaled = stc.transform(X_test_ohe)

number_of_features = X_train_scaled.shape[1]  # Number of features in the input data

# Define the classifiers
seed = 42
models = [
    ('LR', LogisticRegression(solver='lbfgs', random_state=seed, max_iter=1000)),
    ('DT', DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=seed)),
    ('RF', RandomForestClassifier(n_estimators=200, max_depth=5, random_state=seed, min_samples_leaf=3)),
    ('KNN', KNeighborsClassifier(n_neighbors=3)),
    ('XGB', XGBClassifier(eval_metric='logloss', use_label_encoder=False)),
    ('SVM', SVC(gamma='scale', random_state=seed, probability=True)),
    ('MLP', KerasClassifier(build_fn=lambda: create_network(number_of_features), epochs=100, batch_size=100, verbose=0))
]

# Train and evaluate the models
for name, model in models:
    start_time = time.time()  # Start time
    
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    y_pred_prob = model.predict_proba(X_test_scaled)
    
    end_time = time.time()  # End time
    computation_time = end_time - start_time  # Computation time
    
    print(f"Classifier: {name}")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred, digits=3))  # Set digits to 3 for three decimal places
    print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred_prob[:, 1]):.3f}")
    print(f"Computation Time: {computation_time:.3f} seconds")
    print()

C:\Users\xg16060\Miniconda3\envs\survey_paper\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\xg16060\AppData\Local\Temp\1\ipykernel_29380\1600309740.py:60: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('MLP', KerasClassifier(build_fn=lambda: create_network(number_of_features), epochs=100, batch_size=100, verbose=0))


Classifier: LR
[[6320  498]
 [ 962 1265]]
              precision    recall  f1-score   support

           0      0.868     0.927     0.896      6818
           1      0.718     0.568     0.634      2227

    accuracy                          0.839      9045
   macro avg      0.793     0.747     0.765      9045
weighted avg      0.831     0.839     0.832      9045

ROC AUC Score: 0.894
Computation Time: 0.049 seconds

Classifier: DT
[[6453  365]
 [1089 1138]]
              precision    recall  f1-score   support

           0      0.856     0.946     0.899      6818
           1      0.757     0.511     0.610      2227

    accuracy                          0.839      9045
   macro avg      0.806     0.729     0.754      9045
weighted avg      0.831     0.839     0.828      9045

ROC AUC Score: 0.857
Computation Time: 0.029 seconds

Classifier: RF
[[6522  296]
 [1081 1146]]
              precision    recall  f1-score   support

           0      0.858     0.957     0.905      6818
   

In [2]:
import config_cat_embedding
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
#from keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from tqdm import tqdm_notebook as tqdm

from data_prep import bank_data_prep, adult_data_prep
from embedding_helper import create_network
import time
import random

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone


# Set the random seed for reproducibility
random.seed(42)
np.random.seed(42)

data_path = config_cat_embedding.paths['data']
data_path_out = config_cat_embedding.paths['data_output']

bank_data = pd.read_csv(data_path+'adult.csv', sep=',')

df_bank, cat_cols = adult_data_prep(bank_data)


X = df_bank.iloc[:, :-1]
target = df_bank.y

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=1500)

# One-hot encoding
import category_encoders as ce
target_encoder = ce.TargetEncoder(cols=cat_cols,smoothing=1)
X_train_ohe = target_encoder.fit_transform(X_train, y_train)
X_test_ohe = target_encoder.transform(X_test)  # Use transform() instead of fit_transform()

# Standard scaling
stc = StandardScaler()
X_train_scaled = stc.fit_transform(X_train_ohe)
X_test_scaled = stc.transform(X_test_ohe)

number_of_features = X_train_scaled.shape[1]  # Number of features in the input data

# Define the classifiers
seed = 42
models = [
    ('LR', LogisticRegression(solver='lbfgs', random_state=seed, max_iter=1000)),
    ('DT', DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=seed)),
    ('RF', RandomForestClassifier(n_estimators=200, max_depth=5, random_state=seed, min_samples_leaf=3)),
    ('KNN', KNeighborsClassifier(n_neighbors=3)),
    ('XGB', XGBClassifier(eval_metric='logloss', use_label_encoder=False)),
    ('SVM', SVC(gamma='scale', random_state=seed, probability=True)),
    ('MLP', KerasClassifier(build_fn=lambda: create_network(number_of_features), epochs=100, batch_size=100, verbose=0))
]


# Define number of runs for cross-validation
n_runs = 5
kf = StratifiedKFold(n_splits=n_runs, shuffle=True, random_state=seed)

# Initialize dictionary to store results
results = {}

# Loop over the models
for name, model in models:
    print(f"Classifier: {name}")
    
    # Lists to store metrics for each fold
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    roc_auc_scores = []
    computation_times = []
    
    # Perform cross-validation
    for train_idx, test_idx in kf.split(X_train_scaled, y_train):
        # Clone the model to ensure fresh training for each fold
        clf = clone(model)
        
        X_train_fold, X_test_fold = X_train_scaled[train_idx], X_train_scaled[test_idx]
        y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]
        
        start_time = time.time()  # Start time
        
        clf.fit(X_train_fold, y_train_fold)
        y_pred = clf.predict(X_test_fold)
        y_pred_prob = clf.predict_proba(X_test_fold)[:, 1]  # Probabilities for ROC AUC
        
        end_time = time.time()  # End time
        computation_time = end_time - start_time  # Computation time
        
        # Append metrics for this fold
        accuracy_scores.append(accuracy_score(y_test_fold, y_pred))
        precision_scores.append(precision_score(y_test_fold, y_pred))
        recall_scores.append(recall_score(y_test_fold, y_pred))
        f1_scores.append(f1_score(y_test_fold, y_pred))
        roc_auc_scores.append(roc_auc_score(y_test_fold, y_pred_prob))
        computation_times.append(computation_time)
    
    # Compute mean and standard deviation of all metrics
    mean_accuracy = np.mean(accuracy_scores)
    std_accuracy = np.std(accuracy_scores)
    
    mean_precision = np.mean(precision_scores)
    std_precision = np.std(precision_scores)
    
    mean_recall = np.mean(recall_scores)
    std_recall = np.std(recall_scores)
    
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)
    
    mean_roc_auc = np.mean(roc_auc_scores)
    std_roc_auc = np.std(roc_auc_scores)
    
    mean_time = np.mean(computation_times)
    std_time = np.std(computation_times)
    
    # Store results for the current model
    results[name] = {
        "Mean Accuracy": mean_accuracy, "STD Accuracy": std_accuracy,
        "Mean Precision": mean_precision, "STD Precision": std_precision,
        "Mean Recall": mean_recall, "STD Recall": std_recall,
        "Mean F1 Score": mean_f1, "STD F1 Score": std_f1,
        "Mean ROC AUC": mean_roc_auc, "STD ROC AUC": std_roc_auc,
        "Mean Time": mean_time, "STD Time": std_time
    }
    
    # Print results
    print(f"Mean Accuracy: {mean_accuracy:.3f} ± {std_accuracy:.3f}")
    print(f"Mean Precision: {mean_precision:.3f} ± {std_precision:.3f}")
    print(f"Mean Recall: {mean_recall:.3f} ± {std_recall:.3f}")
    print(f"Mean F1 Score: {mean_f1:.3f} ± {std_f1:.3f}")
    print(f"Mean ROC AUC: {mean_roc_auc:.3f} ± {std_roc_auc:.3f}")
    print(f"Mean Computation Time: {mean_time:.3f} ± {std_time:.3f} seconds")
    print()



Classifier: LR
Mean Accuracy: 0.845 ± 0.005
Mean Precision: 0.742 ± 0.011
Mean Recall: 0.578 ± 0.018
Mean F1 Score: 0.650 ± 0.013
Mean ROC AUC: 0.900 ± 0.004
Mean Computation Time: 0.040 ± 0.005 seconds

Classifier: DT
Mean Accuracy: 0.840 ± 0.004
Mean Precision: 0.768 ± 0.013
Mean Recall: 0.512 ± 0.020
Mean F1 Score: 0.614 ± 0.014
Mean ROC AUC: 0.862 ± 0.005
Mean Computation Time: 0.058 ± 0.005 seconds

Classifier: RF
Mean Accuracy: 0.853 ± 0.004
Mean Precision: 0.814 ± 0.006
Mean Recall: 0.530 ± 0.016
Mean F1 Score: 0.642 ± 0.013
Mean ROC AUC: 0.907 ± 0.004
Mean Computation Time: 3.123 ± 0.130 seconds

Classifier: KNN
Mean Accuracy: 0.823 ± 0.002
Mean Precision: 0.657 ± 0.003
Mean Recall: 0.601 ± 0.012
Mean F1 Score: 0.628 ± 0.007
Mean ROC AUC: 0.829 ± 0.006
Mean Computation Time: 7.121 ± 0.486 seconds

Classifier: XGB


C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\xgboost\core.py:158: UserWarning: [22:30:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\xgboost\core.py:158: UserWarning: [22:30:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\xgboost\core.py:158: UserWarning: [22:30:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used

Mean Accuracy: 0.868 ± 0.005
Mean Precision: 0.772 ± 0.007
Mean Recall: 0.662 ± 0.019
Mean F1 Score: 0.713 ± 0.014
Mean ROC AUC: 0.925 ± 0.003
Mean Computation Time: 0.270 ± 0.012 seconds

Classifier: SVM
Mean Accuracy: 0.851 ± 0.005
Mean Precision: 0.766 ± 0.011
Mean Recall: 0.574 ± 0.019
Mean F1 Score: 0.656 ± 0.014
Mean ROC AUC: 0.879 ± 0.004
Mean Computation Time: 777.781 ± 17.904 seconds

Classifier: MLP


C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\keras\src\layers\core\dense.py:87

Mean Accuracy: 0.846 ± 0.003
Mean Precision: 0.724 ± 0.034
Mean Recall: 0.623 ± 0.067
Mean F1 Score: 0.666 ± 0.025
Mean ROC AUC: 0.905 ± 0.002
Mean Computation Time: 68.883 ± 6.507 seconds

